In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBRanker
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv(".\\csv_파일\\16000개_데이터.csv")

In [3]:
# llm_rank를 관련성 점수로 변환
# 1위 -> 3점, 2위 -> 2점, 3위 -> 1점, 나머지는 0점 (역순 변환)
def convert_rank_to_score(rank):
    if rank == 1: return 3
    elif rank == 2: return 2
    elif rank == 3: return 1
    else: return 0

# 정답(y)은 정수형으로 변환 (0~10 단계로 변환)
df['relevance_label'] = df['llm_rank'].apply(convert_rank_to_score)

---
### 데이터 분할

In [4]:
# 1. 사용자 유형별로 질문(Group) 리스트 추출
queries_0 = df[df['사용자 유형'] == 0]['질문'].unique() # 입문자 질문들
queries_1 = df[df['사용자 유형'] == 1]['질문'].unique() # 기존 팬 질문들

# 2. 각 유형 내에서 8:2로 학습/테스트 질문 분리
# (전체 데이터에서도 8:2 비율이 유지되면서 테스트셋 구성됨)
train_q0, test_q0 = train_test_split(queries_0, test_size=0.2, random_state=42)
train_q1, test_q1 = train_test_split(queries_1, test_size=0.2, random_state=42)

# 3. 최종 학습 및 테스트 데이터 생성
train_df = df[df['질문'].isin(list(train_q0) + list(train_q1))].sort_values('질문')
test_df  = df[df['질문'].isin(list(test_q0) + list(test_q1))].sort_values('질문')

# 4) ✅ 인코더는 train에만 fit 하고, test는 transform만!
#    + test에 새로운 팀이 나와도 깨지지 않게 unknown_value=-1 처리
enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

train_df[['기존_응원팀_encoded', '매칭팀_encoded']] = enc.fit_transform(
    train_df[['기존 응원 팀', '매칭팀']]
)

test_df[['기존_응원팀_encoded', '매칭팀_encoded']] = enc.transform(
    test_df[['기존 응원 팀', '매칭팀']]
)

# 4. 피처와 정답 정의 (llm_rank 기반 관련성 점수 활용)
features = ['사용자 유형', 'sbert_score', 'n2v_score', 'vector_score' , '기존_응원팀_encoded' , '매칭팀_encoded']
X_train = train_df[features]
y_train = train_df['llm_rank'].apply(lambda x: 4-x if x <= 3 else 0) # 1위->3점, 2위->2점...

X_test = test_df[features]
y_test = test_df['llm_rank'].apply(lambda x: 4-x if x <= 3 else 0)

# 5. 그룹 정보 계산
group_train = train_df.groupby('질문').size().to_list()
group_test = test_df.groupby('질문').size().to_list()

In [5]:
# 스케일러 (RobustScaler)
from sklearn.preprocessing import RobustScaler
# 1) 스케일링할 연속형 컬럼만 지정 (임베딩 유사도/점수 계열)
scale_cols = ['sbert_score', 'n2v_score', 'vector_score']
# 2) RobustScaler 객체 생성 (중앙값/사분위 기반으로 이상치에 강함)
scaler = RobustScaler()
# 3) train 데이터로만 스케일 기준을 학습(fit)하고, 동시에 변환(transform)
train_df[scale_cols] = scaler.fit_transform(train_df[scale_cols])
# 4) test 데이터는 train에서 만든 기준으로만 변환(transform) (데이터 누수 방지)
test_df[scale_cols] = scaler.transform(test_df[scale_cols])
# 5) 스케일링 반영된 피처로 X를 다시 구성
X_train = train_df[features]
X_test = test_df[features]

---
### 모델 학습

In [6]:
model = XGBRanker(
    objective="rank:ndcg",
    eval_metric="ndcg@3",
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

In [7]:
model.fit(X_train, y_train, group = group_train)

,objective,'rank:ndcg'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'ndcg@3'


🔍 LightGBM 로그 분석 결과
1. `No further splits with positive gain` (가장 많이 뜬 경고)
    - 의미: 나무를 더 잘게 쪼개서 학습하려고 했는데, 더 이상 쪼개봤자 성능 향상에 도움이 안 된다고 판단해서 멈춘 거야.

    - 원인: 우리 데이터(약 1,100행)가 머신러닝 기준으로는 아주 큰 편은 아니고, 피처(4개)가 명확해서 모델이 금방 정답 패턴을 찾았기 때문이야.

    - 결과: 이건 에러가 아니라 모델이 과적합(Overfitting)되지 않게 스스로 공부를 적당히 멈춘 똑똑한 상태라고 보면 돼!

2. `Found whitespace in feature_names`
    - 의미: 피처 이름 중에 '사용자 유형'처럼 공백(띄어쓰기)이 있는 게 있다는 뜻이야.

    - 결과: LightGBM이 알아서 사용자_유형처럼 언더바(_)로 바꿔서 처리했으니 아무 문제 없어.

3. `Found 'eval_at' in params`
    - 의미: eval_at 파라미터가 중복으로 설정됐다는 경고야.

    - 결과: 모델 생성할 때 넣은 설정값이 우선 적용됐다는 뜻이니 이것도 안심해도 돼.

In [8]:
scores = model.predict(X_test)

---
### 모델 검증 $\rightarrow$ 실제 추천 리스트로 변환

In [9]:
# 모델이 뱉은 점수를 test_df에 다시 붙여서, 질문별로 어떤 팀이 1등인지 눈으로 확인

# 1. 테스트 데이터 복사본에 예측 점수 추가
test_results = test_df.copy()
test_results['predicted_score'] = scores

# 2. 질문별로 점수가 높은 순(내림차순)으로 정렬
# '질문'과 'predicted_score' 순으로 정렬
test_results = test_results.sort_values(by=['질문', 'predicted_score'], ascending=[True, False])

# 3. 각 질문에서 모델이 뽑은 Top 3 팀 확인
final_top3 = test_results.groupby('질문').head(3)
print(final_top3[['질문', '매칭팀', 'predicted_score', 'llm_rank']])

# '사용자 유형'별로 필터링해서 저장해두면 나중에 피그마 UI에 넣기 좋다고 함
final_top3.to_csv("final_recommendation_results.csv", index=False, encoding="utf-8-sig")

                                                      질문     매칭팀  \
1812                    나는 EPL는 처음인데, 낭만 성향이 강한 팀을 추천해줘.  두샨 베어스   
9941                    나는 EPL는 처음인데, 낭만 성향이 강한 팀을 추천해줘.  두샨 베어스   
6058                    나는 EPL는 처음인데, 낭만 성향이 강한 팀을 추천해줘.     페라리   
1460            나는 EPL는 처음인데, 수비와(과) 자부심 성향이 강한 팀을 추천해줘.  두샨 베어스   
15240           나는 EPL는 처음인데, 수비와(과) 자부심 성향이 강한 팀을 추천해줘.  두샨 베어스   
...                                                  ...     ...   
5746   나는 키움 히어로즈의 '고척돔' 같은 스타일이 마음에 들어. 이런 느낌을 가진 다른...     페라리   
5757   나는 키움 히어로즈의 '고척돔' 같은 스타일이 마음에 들어. 이런 느낌을 가진 다른...   FC 서울   
14327  나는 페라리의 '붉은 군단' 같은 스타일이 마음에 들어. 이런 느낌을 가진 다른 리...  두샨 베어스   
986    나는 페라리의 '붉은 군단' 같은 스타일이 마음에 들어. 이런 느낌을 가진 다른 리...  두샨 베어스   
15514  나는 페라리의 '붉은 군단' 같은 스타일이 마음에 들어. 이런 느낌을 가진 다른 리...   FC 서울   

       predicted_score  llm_rank  
1812          1.873794         0  
9941          1.873794         0  
6058          1.054241         1  
1460          2.357808         0  
15240   

### NDCG
: 추천된 팀들의 **순서(Ranking)**가 얼마나 정확한지 측정하는 지표.
</br> 1등 팀을 1등으로 맞추는 게 2등으로 맞추는 것보다 더 높은 점수를 받음.

- 0.7 이상: 모델이 꽤 영리하게 추천을 잘해줌

- 0.9 이상: LLM이 매긴 순위랑 거의 똑같이 맞추고 있음

- 만약 점수가 너무 낮게(0.5 이하) 나온다면?
</br>: 모델이 아직 '사용자 유형'이나 '벡터 스코어'의 특징을 제대로 못 잡았다는 신호
</br>$\Rightarrow$ learning_rate를 조절하거나 피처를 더 추가하는 파라미터 튜닝 단계 진행

In [ ]:
# 모델 성능 평가하기 (NDCG 지표 확인)

from sklearn.metrics import ndcg_score
import numpy as np

# 1. 질문별 NDCG 점수를 저장할 리스트
ndcg_list = []
ndcg_list2 = []
ndcg_list3 = []
# 2. 질문(그룹)별로 루프를 돌며 계산
for qid in test_results['질문'].unique():
    # 해당 질문에 속한 데이터만 추출
    group_data = test_results[test_results['질문'] == qid]
    
    # 실제 정답(llm_rank 기반 점수)과 모델의 예측 점수 추출
    # ndcg_score는 2차원 배열 형식을 원하므로 reshape(1, -1)을 해줘.
    y_true = np.array([group_data['relevance_label']]).reshape(1, -1)
    y_score = np.array([group_data['predicted_score']]).reshape(1, -1)
    
    # NDCG@3 계산 (상위 3개 팀의 순위가 얼마나 정확한가?)
    # 데이터가 3개보다 적을 경우를 대비해 k값을 조절해.
    k = min(len(group_data), 3)
    k2 = min(len(group_data), 5)
    k3 = min(len(group_data), 10)
    score = ndcg_score(y_true, y_score, k = k)
    score2 = ndcg_score(y_true, y_score, k = k2)
    score3 = ndcg_score(y_true, y_score, k = k3)
    ndcg_list.append(score)
    ndcg_list2.append(score2)
    ndcg_list3.append(score3)

# 3. 최종 평균 NDCG 출력
avg_ndcg = np.mean(ndcg_list)
avg_ndcg2 = np.mean(ndcg_list2)
avg_ndcg3 = np.mean(ndcg_list3)

print("=" * 50)
print(f"NDCG@3 : {avg_ndcg:.2f}")
print(f"NDCG@5 : {avg_ndcg2:.2f}")
print(f"NDCG@10 : {avg_ndcg2:.2f}")
# ================================
# ✅ [ADDED] Hit Rate@K 계산
# ================================
K = 3  # Top-K 기준 (NDCG@3과 맞춤)

hit_list = []
# 질문별 hit 여부(0/1)를 저장할 리스트

for qid in test_results['질문'].unique():
    # 질문 하나씩 반복
    
    group_data = test_results[test_results['질문'] == qid]
    # 해당 질문에 대한 후보 팀들만 가져온다
    
    group_sorted = group_data.sort_values(
        by='predicted_score',
        ascending=False
    )
    # 예측 점수 기준으로 내림차순 정렬 (Top-K 추출용)
    
    top_k = group_sorted.head(K)
    # 상위 K개 팀 선택
    
    hit = int((top_k['relevance_label'] > 0).any())
    # Top-K 안에 정답(relevance_label > 0)이
    # 하나라도 있으면 1, 없으면 0
    
    hit_list.append(hit)
    # 질문 하나의 hit 결과 저장

hit_rate = np.mean(hit_list)
# 전체 질문에 대한 평균 = Hit Rate@K

print(f" 최종 모델 성능 (Hit Rate@{K}): {hit_rate:.4f}")

print("=" * 50)


NDCG@3): 0.37
NDCG@5): 0.43
NDCG@10): 0.43
 최종 모델 성능 (Hit Rate@3): 0.7561


🔍 결과 분석
- 변별력 확인: 가장 좋은 팀($1.38$)과 가장 안 좋은 팀($-0.74$)의 점수 차이가 $2$점 넘게 벌어졌어.
- 논리적 일치: 우리가 설정한 sbert_score, n2v_score, vector_score가 높을수록 predicted_score도 정비례해서 올라가는 걸 보니, 모델이 피처 간의 관계를 아주 정확하게 파악했어.
- 마이너스의 미학: 아까 한 개만 넣었을 때 나왔던 마이너스 점수($-0.75$)가 이제 보니 '비추천팀' 수준의 점수였다는 것도 알게 됐네! 랭킹 모델의 세계에선 아주 흔한 일이니 이제 완전히 안심해도 돼.

사이트와의 연결을 위해 학습된 모델 저장

---
# GridSearchCV

In [11]:
from itertools import product
from sklearn.metrics import ndcg_score
import numpy as np

# 1. 파라미터 그리드 설정
param_grid = {
    "learning_rate": [0.05, 0.1],
    "max_depth": [4, 6],
    "n_estimators": [200, 400],
    "num_leaves": [15, 31], # LightGBM의 핵심 파라미터 추가
    "subsample": [0.8, 1.0]
}

best_score = -1
best_params = None

# 2. 모든 조합에 대해 루프 실행 (itertools.product)
for lr, md, ne, nl, ss in product(
    param_grid["learning_rate"],
    param_grid["max_depth"],
    param_grid["n_estimators"],
    param_grid["num_leaves"],
    param_grid["subsample"]
):
    # LightGBM 랭킹 모델 정의
    model = XGBRanker(
        objective="rank:ndcg",
        learning_rate=lr,
        max_depth=md,
        n_estimators=ne,
        subsample=ss,
        random_state=42
    )

    # 모델 학습 (group 정보 필수!)
    model.fit(
        X_train, y_train,
        group=group_train
    )

    # 예측 및 성능 평가
    preds = model.predict(X_test)
    
    # [주의] ndcg_score는 2차원 배열을 기대하므로 reshape 처리
    # 실제 데이터의 group 구조에 따라 루프를 돌며 평균을 내는 것이 더 정확해!
    score = ndcg_score([y_test], [preds])

    if score > best_score:
        best_score = score
        best_params = {
            "learning_rate": lr,
            "max_depth": md,
            "n_estimators": ne,
            "num_leaves": nl,
            "subsample": ss
        }

print("=" * 50)
print(f" 최종 모델 성능 (Average NDCG@3): Best NDCG: {best_score}")
# print("  Best Params:", best_params)
print("=" * 50)

 최종 모델 성능 (Average NDCG@3): Best NDCG: 0.9059135821446663
